# About: KVMのインストール

ベアメタルマシンに対して、KVMをインストールする。

以下の前提条件に基づいて作成されている。適宜読み替えをおこなうこと。

- OS: CentOS 6 (Notebook作成時は6.7)
- NIC: 外向けネットワークのインタフェース名は *eth1* とし、仮想マシンはeth1を介して直接(NATせずに)外部ネットワークと通信をおこなう
    - VMへのIPアドレス割り当てはdnsmasqを使う

## Operation Note

*ここに経緯を記述*

# Notebookと対象(Baremetalマシン)のBinding

Inventory中のgroup名でBind対象を指示する。

In [1]:
target_group = 'test-hypervisor'

Bind対象への疎通状態を確認する。

In [2]:
!ansible -m ping {target_group}

XXX.XXX.XXX.105 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


## CPUの確認

KVMをインストールしVMを作成、利用するためには、Virtualization Technology (VT)が有効にされている必要がある。

以下のコマンド出力が1行以上であれば(svm, vmxをflagsに含むCPUが1つ以上あれば)OK。

In [3]:
!ansible -m shell -a "egrep '(vmx|svm)' /proc/cpuinfo" {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc arch_perfmon pebs bts rep_good xtopology nonstop_tsc aperfmperf pni pclmulqdq dtes64 monitor ds_cpl vmx smx est tm2 ssse3 fma cx16 xtpr pdcm pcid dca sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand lahf_lm abm ida arat epb xsaveopt pln pts dts tpr_shadow vnmi flexpriority ept vpid fsgsbase bmi1 avx2 smep bmi2 erms invpcid
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc arch_perfmon pebs bts rep_good xtopology nonstop_tsc aperfmperf pni pclmulqdq dtes64 monitor ds_cpl vmx smx est tm2 ssse3 fma cx16 xtpr pdcm pcid dca sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand lahf_lm abm ida arat epb xsaveopt pln 

## NICの設定

このNotebookでは、外部接続されているNICとつながるブリッジを作成し、このブリッジにVMのNICを接続する。

外部接続されているNICの名前を確認しておく。

> お手本NotebookはAICのNIC構成にあわせてあるので、研究クラウド向けには定義を修正すること。

In [4]:
external_nic = 'eth1'

現在の定義を確認しておく・・・現在、Notebook環境からマシンへのSSHに利用している外向けIPアドレスが見えればOK。

In [5]:
!ansible -a '/sbin/ip addr show {external_nic}' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
2: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP qlen 1000
    link/ether XX:XX:XX:XX:XX:XX brd XX:XX:XX:XX:XX:XX
    inet XXX.XXX.XXX.105/26 brd XXX.XXX.XXX.127 scope global eth1
    inet6 XX:XX:XX:XX:XX:XX/64 scope link 
       valid_lft forever preferred_lft forever



## KVMに与える設定情報

このPlaybookではDHCP払い出しに関する情報を定義する必要がある。

- vmips ... VMに割り当てるアドレス範囲。開始、終了をカンマで区切る
- vmgateway ... VMに設定するDefault gateway

> この情報はInventoryに記述したほうが、今後のメンテナンスが楽になるはず・・・

In [6]:
vmips = 'XXX.XXX.XXX.66,XXX.XXX.XXX.66'
vmgateway = 'XXX.XXX.XXX.126'

## OSの確認

このNotebookは、**CentOS 6がインストールされた環境にBindingされることを前提として実装**している。

以下のコマンドの出力が **Distributor ID: CentOS**, **Release: 6.x** であることを確認する。

In [7]:
!ansible -a 'lsb_release -a' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
LSB Version:	:base-4.0-amd64:base-4.0-noarch:core-4.0-amd64:core-4.0-noarch:graphics-4.0-amd64:graphics-4.0-noarch:printing-4.0-amd64:printing-4.0-noarch
Distributor ID:	CentOS
Description:	CentOS release 6.7 (Final)
Release:	6.7
Codename:	Final



# KVMのインストール

事前条件の確認ができたので、KVMをインストールしていく。

## CentOSのパッケージ更新

念のため、パッケージを更新しておく。

In [8]:
!ansible -b -m yum -a 'name=* state=latest' {target_group}

XXX.XXX.XXX.105 | SUCCESS => {
    "changed": true, 
    "msg": "", 
    "rc": 0, 
    "results": [
        "Loaded plugins: fastestmirror\nSetting up Update Process\nLoading mirror speeds from cached hostfile\n * base: www.ftp.ne.jp\n * extras: www.ftp.ne.jp\n * updates: www.ftp.ne.jp\nResolving Dependencies\n--> Running transaction check\n---> Package ConsoleKit.x86_64 0:0.4.1-3.el6 will be updated\n---> Package ConsoleKit.x86_64 0:0.4.1-6.el6 will be an update\n---> Package ConsoleKit-libs.x86_64 0:0.4.1-3.el6 will be updated\n---> Package ConsoleKit-libs.x86_64 0:0.4.1-6.el6 will be an update\n---> Package alsa-lib.x86_64 0:1.0.22-3.el6 will be updated\n---> Package alsa-lib.x86_64 0:1.1.0-4.el6 will be an update\n---> Package audit.x86_64 0:2.3.7-5.el6 will be updated\n---> Package audit.x86_64 0:2.4.5-3.el6 will be an update\n---> Package audit-libs.x86_64 0:2.3.7-5.el6 will be updated\n---> Package audit-libs.x86_64 0:2.4.5-3.el6 will be an update\n---> Package bash.x86_64 0:4.1

念のため更新後のOSバージョンも確認しておく。

In [9]:
!ansible -a 'lsb_release -a' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
LSB Version:	:base-4.0-amd64:base-4.0-noarch:core-4.0-amd64:core-4.0-noarch:graphics-4.0-amd64:graphics-4.0-noarch:printing-4.0-amd64:printing-4.0-noarch
Distributor ID:	CentOS
Description:	CentOS release 6.8 (Final)
Release:	6.8
Codename:	Final



## 仮想マシン用ブリッジの準備

仮想マシン用のブリッジ設定を追加する。

In [10]:
import tempfile
temp_dir = tempfile.mkdtemp()
temp_dir

'/tmp/tmpWOb3Yh'

ブリッジ定義を修正するPlaybookを作る。以下の処理をおこなう。external_nicがeth1の場合は・・・

- ブリッジ br-eth1 インタフェースを作成 ... ここには現在のeth1のIPアドレスを設定
- eth1インタフェースを修正 ... IPアドレスの情報を削除し、Promiscuousとしてbr-eth1インタフェースに接続

In [11]:
bridge_nic = 'br-' + external_nic

In [12]:
import os

with open(os.path.join(temp_dir, 'bridge.yml'), 'w') as f:
    f.write('''- hosts: {hosts}
  become: yes
  handlers:
    - name: Restart network
      service: name=network state=restarted
  tasks:
    - name: Install bridge-utils
      yum: name=bridge-utils state=latest
  
    - name: Create {br}
      command: cp /etc/sysconfig/network-scripts/ifcfg-{nic} /etc/sysconfig/network-scripts/ifcfg-{br} creates=/etc/sysconfig/network-scripts/ifcfg-{br}
      
    - name: Modify {nic}
      lineinfile: dest=/etc/sysconfig/network-scripts/ifcfg-{nic} regexp='^IPADDR=' state=absent
      notify: Restart network
      
    - name: Modify {nic}
      lineinfile: dest=/etc/sysconfig/network-scripts/ifcfg-{nic} regexp='^NETMASK=' state=absent
      notify: Restart network
      
    - name: Modify {nic}
      lineinfile: dest=/etc/sysconfig/network-scripts/ifcfg-{nic} regexp='^GATEWAY=' state=absent
      notify: Restart network
      
    - name: Modify {nic}
      lineinfile: dest=/etc/sysconfig/network-scripts/ifcfg-{nic} regexp='^PROMISC=' line='PROMISC="yes"' state=present
      notify: Restart network
      
    - name: Modify {nic}
      lineinfile: dest=/etc/sysconfig/network-scripts/ifcfg-{nic} regexp='^BRIDGE=' line='BRIDGE="{br}"' state=present
      notify: Restart network
      
    - name: Modify {br}
      lineinfile: dest=/etc/sysconfig/network-scripts/ifcfg-{br} regexp='^DEVICE=' line='DEVICE="{br}"' state=present
      notify: Restart network
      
    - name: Modify {br}
      lineinfile: dest=/etc/sysconfig/network-scripts/ifcfg-{br} regexp='^TYPE=' line='TYPE="Bridge"' state=present
      notify: Restart network
'''.format(hosts=target_group, nic=external_nic, br=bridge_nic))

!cat {temp_dir}/bridge.yml

- hosts: test-hypervisor
  become: yes
  handlers:
    - name: Restart network
      service: name=network state=restarted
  tasks:
    - name: Install bridge-utils
      yum: name=bridge-utils state=latest
  
    - name: Create br-eth1
      command: cp /etc/sysconfig/network-scripts/ifcfg-eth1 /etc/sysconfig/network-scripts/ifcfg-br-eth1 creates=/etc/sysconfig/network-scripts/ifcfg-br-eth1
      
    - name: Modify eth1
      lineinfile: dest=/etc/sysconfig/network-scripts/ifcfg-eth1 regexp='^IPADDR=' state=absent
      notify: Restart network
      
    - name: Modify eth1
      lineinfile: dest=/etc/sysconfig/network-scripts/ifcfg-eth1 regexp='^NETMASK=' state=absent
      notify: Restart network
      
    - name: Modify eth1
      lineinfile: dest=/etc/sysconfig/network-scripts/ifcfg-eth1 regexp='^GATEWAY=' state=absent
      notify: Restart network
      
    - name: Modify eth1
      lineinfile: dest=/etc/sysconfig/network-scripts/ifcfg-eth1 regexp='^PROMISC=' line='PROMISC="ye

念のため、check modeで動作させてみる。

In [13]:
!ansible-playbook -CDv {temp_dir}/bridge.yml

Using /etc/ansible/ansible.cfg as config file

PLAY [test-hypervisor] *********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.105]

TASK [Install bridge-utils] ****************************************************
changed: [XXX.XXX.XXX.105] => {"changed": true, "changes": {"installed": ["bridge-utils"], "updated": []}, "msg": "", "rc": 0, "results": []}

TASK [Create br-eth1] **********************************************************
skipping: [XXX.XXX.XXX.105] => {"changed": false, "msg": "remote module does not support check mode", "skipped": true}

TASK [Modify eth1] *************************************************************
changed: [XXX.XXX.XXX.105] => {"backup": "", "changed": true, "found": 1, "msg": "1 line(s) removed"}
--- before: /etc/sysconfig/network-scripts/ifcfg-eth1 (content)
+++ after: /etc/sysconfig/network-scripts/ifcfg-eth1 (content)
@@ -2,7 +2,6 @@
 BOOTPROTO="stat

変更対象が意図したホストであることを確認して、実行する。

In [14]:
!ansible-playbook {temp_dir}/bridge.yml


PLAY [test-hypervisor] *********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.105]

TASK [Install bridge-utils] ****************************************************
changed: [XXX.XXX.XXX.105]

TASK [Create br-eth1] **********************************************************
changed: [XXX.XXX.XXX.105]

TASK [Modify eth1] *************************************************************
changed: [XXX.XXX.XXX.105]

TASK [Modify eth1] *************************************************************
changed: [XXX.XXX.XXX.105]

TASK [Modify eth1] *************************************************************
changed: [XXX.XXX.XXX.105]

TASK [Modify eth1] *************************************************************
changed: [XXX.XXX.XXX.105]

TASK [Modify eth1] *************************************************************
changed: [XXX.XXX.XXX.105]

TASK [Modify br-eth1] ****************************


操作用のIPアドレスはブリッジに設定している。念のため各インタフェースの状況を確認しておく。

ブリッジインタフェースが存在し、もともと外向きNICに割り当てられていたIPアドレスが設定されていればOK。

In [15]:
!ansible -a "/sbin/ip addr show {bridge_nic}" {target_group}
!ansible -a "/usr/sbin/brctl show {bridge_nic}" {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
10: br-eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UNKNOWN 
    link/ether XX:XX:XX:XX:XX:XX brd XX:XX:XX:XX:XX:XX
    inet XXX.XXX.XXX.105/26 brd XXX.XXX.XXX.127 scope global br-eth1
    inet6 XX:XX:XX:XX:XX:XX/64 scope link 
       valid_lft forever preferred_lft forever

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
bridge name	bridge id		STP enabled	interfaces
br-eth1		8000.246e960db538	no		eth1



現在の外向きインタフェースの情報も確認しておく。

In [16]:
!ansible -a "/sbin/ip addr show {external_nic}" {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
2: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP qlen 1000
    link/ether XX:XX:XX:XX:XX:XX brd XX:XX:XX:XX:XX:XX
    inet6 XX:XX:XX:XX:XX:XX/64 scope link 
       valid_lft forever preferred_lft forever



## パッケージのインストール

KVMのインストールはyumでおこなう。

In [17]:
!mkdir -p {temp_dir}/roles/kvm-host/tasks

In [18]:
%%writefile {temp_dir}/roles/kvm-host/tasks/main.yml
- name: Install required packages
  yum:
    name: "{{ item }}"
  with_items:
    - qemu-kvm
    - qemu-img
    - libvirt
    - libvirt-python
    - libguestfs-tools
    - virt-install
    - virt-top

- name: Start libvirtd service
  service: name=libvirtd state=started enabled=yes

- name: Install virt-manager
  yum: name=virt-manager

Writing /tmp/tmpWOb3Yh/roles/kvm-host/tasks/main.yml


In [19]:
!tree {temp_dir}/roles

/tmp/tmpWOb3Yh/roles
└── kvm-host
    └── tasks
        └── main.yml

2 directories, 1 file


In [20]:
with open(os.path.join(temp_dir, 'kvm.yml'), 'w') as f:
    f.write('''- hosts: {hosts}
  become: yes
  roles:
    - role: kvm-host
'''.format(hosts=target_group))

!cat {temp_dir}/kvm.yml

- hosts: test-hypervisor
  become: yes
  roles:
    - role: kvm-host


念のため、 check modeで動作させてみる。

In [21]:
!ansible-playbook -CDv {temp_dir}/kvm.yml

Using /etc/ansible/ansible.cfg as config file

PLAY [test-hypervisor] *********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.105]

TASK [kvm-host : Install required packages] ************************************
changed: [XXX.XXX.XXX.105] => (item=[u'qemu-kvm', u'qemu-img', u'libvirt', u'libvirt-python', u'libguestfs-tools', u'virt-install', u'virt-top']) => {"changed": true, "changes": {"installed": ["qemu-kvm", "qemu-img", "libvirt", "libvirt-python", "libguestfs-tools", "virt-install", "virt-top"]}, "item": ["qemu-kvm", "qemu-img", "libvirt", "libvirt-python", "libguestfs-tools", "virt-install", "virt-top"], "results": []}

TASK [kvm-host : Start libvirtd service] ***************************************
fatal: [XXX.XXX.XXX.105]: FAILED! => {"changed": false, "failed": true, "msg": "no service or tool found for: libvirtd"}

NO MORE HOSTS LEFT *****************************************

変更対象が意図したホストであることを確認して、実行する。

In [22]:
!ansible-playbook {temp_dir}/kvm.yml


PLAY [test-hypervisor] *********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.105]

TASK [kvm-host : Install required packages] ************************************
changed: [XXX.XXX.XXX.105] => (item=[u'qemu-kvm', u'qemu-img', u'libvirt', u'libvirt-python', u'libguestfs-tools', u'virt-install', u'virt-top'])

TASK [kvm-host : Start libvirtd service] ***************************************
changed: [XXX.XXX.XXX.105]

TASK [kvm-host : Install virt-manager] *****************************************
changed: [XXX.XXX.XXX.105]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.105              : ok=4    changed=3    unreachable=0    failed=0   



## マシンの再起動

インストールで特にエラーなどが起きなければ、再起動する。

参考: http://d.hatena.ne.jp/incarose86/20150215/1424017177

In [23]:
with open(os.path.join(temp_dir, 'reboot.yml'), 'w') as f:
    f.write('''- hosts: {hosts}
  tasks:
    - name: test connection (before reboot)
      ping:

    - name: reboot!
      command: shutdown -r now

    - name: wait for SSH port down
      local_action: wait_for host={{{{ inventory_hostname }}}} port=22 state=stopped

    - name: wait for SSH port up
      wait_for: host={{{{ inventory_hostname }}}} port=22 state=started delay=30
      delegate_to: XXX.XXX.XXX.1

    - name: test connection (after reboot)
      ping:
'''.format(hosts=target_group))

!cat {temp_dir}/reboot.yml

- hosts: test-hypervisor
  tasks:
    - name: test connection (before reboot)
      ping:

    - name: reboot!
      command: shutdown -r now

    - name: wait for SSH port down
      local_action: wait_for host={{ inventory_hostname }} port=22 state=stopped

    - name: wait for SSH port up
      wait_for: host={{ inventory_hostname }} port=22 state=started delay=30
      delegate_to: XXX.XXX.XXX.1

    - name: test connection (after reboot)
      ping:


In [24]:
!ansible-playbook -b {temp_dir}/reboot.yml


PLAY [test-hypervisor] *********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.105]

TASK [test connection (before reboot)] *****************************************
ok: [XXX.XXX.XXX.105]

TASK [reboot!] *****************************************************************
changed: [XXX.XXX.XXX.105]

TASK [wait for SSH port down] **************************************************
ok: [XXX.XXX.XXX.105 -> localhost]

TASK [wait for SSH port up] ****************************************************
ok: [XXX.XXX.XXX.105 -> XXX.XXX.XXX.1]

TASK [test connection (after reboot)] ******************************************
ok: [XXX.XXX.XXX.105]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.105              : ok=6    changed=1    unreachable=0    failed=0   



特にエラーなど起きなければ、引き続き設定を進めていく。

## VMへのDHCP設定

VMにIPアドレスを払いだすため、独自のdnsmasqの設定をおこなっている。

### libvirtのNetwork設定の調整

VMへのアドレス払い出しはここで独自に設定したdnsmasqで直接行いたい。libvirtのdefault network設定は使いたくないので、削除する。

参考: http://dotnsf.blog.jp/archives/2751643.html

In [25]:
!ansible -b -a 'virsh net-destroy default' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
Network default destroyed



再起動時に自動作成されないよう設定する。

In [26]:
!ansible -b -a 'virsh net-autostart default --disable' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
Network default unmarked as autostarted



defaultのstateがinactiveになっていて、かつautostartがnoになっていることを確認する。

In [27]:
!ansible -b -a 'virsh net-list --all' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
Name                 State      Autostart     Persistent
--------------------------------------------------
default              inactive   no            yes



### IPアドレス配布用dnsmasqの準備

VMへのIPアドレス設定はdnsmasqでおこなう。

IPアドレスの払い出しは、自身が持つVMからのみ受け取り、外部からのDHCP requestは拒否する設定をする。

In [28]:
with open(os.path.join(temp_dir, 'vmnetwork.yml'), 'w') as f:
    f.write('''- hosts: {hosts}
  handlers:
    - name: Save iptables
      command: service iptables save

    - name: Reload sysctl
      command: /sbin/sysctl -p

    - name: Start iptables
      service: name=iptables state=started enabled=yes

  tasks:
    - name: Modify dnsmasq
      lineinfile: dest=/etc/dnsmasq.conf regexp='^interface=' line='interface={br}' state=present

    - name: Modify dnsmasq
      lineinfile: dest=/etc/dnsmasq.conf regexp='^dhcp-range=' line='dhcp-range={{{{ vmips }}}},12h' state=present

    - name: Modify dnsmasq
      lineinfile: dest=/etc/dnsmasq.conf regexp='^dhcp-option=' line='dhcp-option=option:router,{{{{ vmgateway }}}}' state=present

    - name: Enabling iptables for bridge
      lineinfile: dest=/etc/sysctl.conf regexp='^{{{{ item }}}}' line='{{{{ item }}}} = 1' state=present
      with_items:
        - 'net.bridge.bridge-nf-call-ip6tables' 
        - 'net.bridge.bridge-nf-call-iptables'
        - 'net.bridge.bridge-nf-call-arptables'
      notify: Reload sysctl

    - name: Configure iptables 
      command: iptables -A INPUT -m physdev --physdev-in {nic} -p {{{{ item }}}} --dport 67:68 -j DROP
      with_items:
        - 'udp'
        - 'tcp'
      notify:
        - Save iptables
        - Start iptables

    - name: Start dnsmasq
      service: name=dnsmasq state=started enabled=yes
'''.format(hosts=target_group, nic=external_nic, br=bridge_nic))

!cat {temp_dir}/vmnetwork.yml

- hosts: test-hypervisor
  handlers:
    - name: Save iptables
      command: service iptables save

    - name: Reload sysctl
      command: /sbin/sysctl -p

    - name: Start iptables
      service: name=iptables state=started enabled=yes

  tasks:
    - name: Modify dnsmasq
      lineinfile: dest=/etc/dnsmasq.conf regexp='^interface=' line='interface=br-eth1' state=present

    - name: Modify dnsmasq
      lineinfile: dest=/etc/dnsmasq.conf regexp='^dhcp-range=' line='dhcp-range={{ vmips }},12h' state=present

    - name: Modify dnsmasq
      lineinfile: dest=/etc/dnsmasq.conf regexp='^dhcp-option=' line='dhcp-option=option:router,{{ vmgateway }}' state=present

    - name: Enabling iptables for bridge
      lineinfile: dest=/etc/sysctl.conf regexp='^{{ item }}' line='{{ item }} = 1' state=present
      with_items:
        - 'net.bridge.bridge-nf-call-ip6tables' 
        - 'net.bridge.bridge-nf-call-iptables'
        - 'net.bridge.bridge-nf-call-arptables'
      notify: Reload sysct

実行してみる・・・

In [29]:
!ansible-playbook -CDv -b -e vmips={vmips} -e vmgateway={vmgateway} {temp_dir}/vmnetwork.yml

Using /etc/ansible/ansible.cfg as config file

PLAY [test-hypervisor] *********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.105]

TASK [Modify dnsmasq] **********************************************************
changed: [XXX.XXX.XXX.105] => {"backup": "", "changed": true, "msg": "line added"}
--- before: /etc/dnsmasq.conf (content)
+++ after: /etc/dnsmasq.conf (content)
@@ -533,3 +533,4 @@
 # Include a another lot of configuration options.
 #conf-file=/etc/dnsmasq.more.conf
 #conf-dir=/etc/dnsmasq.d
+interface=br-eth1


TASK [Modify dnsmasq] **********************************************************
changed: [XXX.XXX.XXX.105] => {"backup": "", "changed": true, "msg": "line added"}
--- before: /etc/dnsmasq.conf (content)
+++ after: /etc/dnsmasq.conf (content)
@@ -533,3 +533,4 @@
 # Include a another lot of configuration options.
 #conf-file=/etc/dnsmasq.more.conf
 #conf-dir=/etc/dnsmas

実施対象を確認した。問題なさそうなので実行する。

In [30]:
!ansible-playbook -b -e vmips={vmips} -e vmgateway={vmgateway} {temp_dir}/vmnetwork.yml


PLAY [test-hypervisor] *********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.105]

TASK [Modify dnsmasq] **********************************************************
changed: [XXX.XXX.XXX.105]

TASK [Modify dnsmasq] **********************************************************
changed: [XXX.XXX.XXX.105]

TASK [Modify dnsmasq] **********************************************************
changed: [XXX.XXX.XXX.105]

TASK [Enabling iptables for bridge] ********************************************
changed: [XXX.XXX.XXX.105] => (item=net.bridge.bridge-nf-call-ip6tables)
changed: [XXX.XXX.XXX.105] => (item=net.bridge.bridge-nf-call-iptables)
changed: [XXX.XXX.XXX.105] => (item=net.bridge.bridge-nf-call-arptables)

TASK [Configure iptables] ******************************************************
changed: [XXX.XXX.XXX.105] => (item=udp)
changed: [XXX.XXX.XXX.105] => (item=tcp)

TASK [Start dnsmasq] *

dnsmasqは実行されているか？

In [31]:
!ansible -b -a 'service dnsmasq status' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
dnsmasq (pid  9388) is running...dnsdomainname: Unknown host



**dnsmasq (pid  XXXXX) is running** と表示されればOK。

# 動作確認

libvirtがインストールされているか？確認・・・

In [32]:
!ansible -b -a 'virsh list' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
 Id    Name                           State
----------------------------------------------------



特にエラーにはならない。OK。

# 後始末

一時ディレクトリを削除する。

In [33]:
!rm -fr {temp_dir}